In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

In [44]:
def loadDataframe(soup, df):
    doctor_dict = {}
    for doctors in soup.find_all('div', {'class':'card dl mb-3'}):
        for i, doctor_data in enumerate(doctors.find_all('div', {'class':'col-sm-4 py-3'})):
            if(i == 0):
                if doctor_data.find('a'):
                    doctor_dict['name'] = doctor_data.find('a').get_text()
                if doctor_data.find('span'):
                    doctor_dict['expertise'] = doctor_data.find('span').get_text()
            elif(i == 1):
                if doctor_data:
                    doctor_info = doctor_data.get_text()
                    doctor_info_split = doctor_info.split('\n')
                    doctor_dict['distance'] = doctor_info_split[0]
                    doctor_dict['street_and_house'] = doctor_info_split[1]
                    doctor_dict['telephone'] = doctor_info_split[3]
                    return(doctor_info)
            doctor_df = pd.DataFrame(doctor_dict, index=[0])

            df = pd.concat([df, doctor_df], ignore_index=True)

            doctor_dict = {}
    
    return(df)

In [7]:
# https://www.arzt-auskunft.de/arzt-auskunft/suche_sn/index.js?a=DL&Mt=&Ft=Allgemeinmedizin&Ft_e=CatId1%3A%3AAllgemeinmedizin%3A%3A2%3B&Ftg=M%C3%BCnchen&Ftg_e=CatId9%3A%3AM%C3%BCnchen%3A%3A48.1471264997798%3A%3A11.552018558777%3B&Otn1=&Otn2=&Sl=&Sd=&St=&Ic1=&Lng=&Db=1
# https://www.arzt-auskunft.de/arzt-auskunft/suche_sn/index.js?a=DL&Ft=Allgemeinmedizin&Ft_e=CatId1%3A%3AAllgemeinmedizin%3A%3A2%3B&Ftg=81377

specializations = [
    'Allgemeinmedizin'
]

locations = [
    '81377'
]

df = pd.DataFrame()

for spl in tqdm(specializations): # desc=f"Running specialization: {spl}"):
    for location in tqdm(locations): # desc=f"Running location: {location}"):
        payload = {'a':'DL', 'Ft':spl, 'Ft_e':f'CatId1%3A%3A{spl}%3A%3A2%3B', 'Ftg':location}
        r = requests.get('https://www.arzt-auskunft.de/arzt-auskunft/suche_sn/index.js', params=payload)
        if(r.status_code == 200):
            soup = BeautifulSoup(r.text, 'html.parser')
            # print(soup)
            loadDataframe(soup, df)
        else:
            print('Something went wrong, please check!')


100%|██████████| 1/1 [00:02<00:00,  2.59s/it]

<!DOCTYPE html>

<html lang="de">
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-V81SSK3CRT"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());

    gtag('config', 'G-V81SSK3CRT');
    </script>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Ärzte, Zahnärzte und Kliniken in der Nähe finden: Die deutschlandweite Arztsuche der Stiftung Gesundheit." name="description"/>
<meta content="Stiftung Gesundheit - Wissen ist die beste Medizin" name="author"/>
<meta content="noindex,follow" name="robots"/>
<title>Die Arztsuche der Stiftung Gesundheit - Arzt-Auskunft</title>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s

In [45]:
doctor_info = loadDataframe(soup, df)
doctor_info_split = doctor_info.split('\n')
doctor_info_split[0]
doctor_info_split[1]
doctor_info_split[3]

'\xa0\xa0\xa0\xa081375 München'

In [46]:
print(doctor_info)


 0.2 km
 Würmtalstraße 18 a     
    81375 München

 089 / 71 77 44






In [61]:
splited = doctor_info.split('\n')
#print(splited)
for el in splited:
    if el.strip('\r'):
        print(el.strip())
#print(splited[1], splited[2], splited[3], splited[5])


0.2 km
Würmtalstraße 18 a
81375 München
089 / 71 77 44
